# **Import modules**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Download data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Fake_news_detection/

/content/drive/MyDrive/Fake_news_detection


In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mustafaosama22","key":"1709b943c6d92c28b8dce0f5e85b79e5"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c fake-news

 71% 33.0M/46.5M [00:00<00:00, 137MB/s]
100% 46.5M/46.5M [00:00<00:00, 158MB/s]


In [ ]:
!unzip /content/drive/MyDrive/Fake_news_detection/fake-news.zip

Archive:  /content/drive/MyDrive/Fake_news_detection/fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


# **Exploring data**

In [ ]:
data = pd.read_csv('train.csv')
data

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [ ]:
print(data.text[0])

House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) 
With apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. 
As we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emails 

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


# **Drop unneeded columns**

In [ ]:
data = data.drop(['id', 'title', 'author'], axis=1)
data

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1
...,...,...
20795,Rapper T. I. unloaded on black celebrities who...,0
20796,When the Green Bay Packers lost to the Washing...,0
20797,The Macy’s of today grew from the union of sev...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",1


# **Handling unknown values**

In [ ]:
data.isnull().sum()

text     39
label     0
dtype: int64

In [ ]:
data = data.dropna()
data

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1
...,...,...
20795,Rapper T. I. unloaded on black celebrities who...,0
20796,When the Green Bay Packers lost to the Washing...,0
20797,The Macy’s of today grew from the union of sev...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",1


# **Data cleaning**

In [ ]:
def cleanText(text):
    text=re.sub(r'#\S*','',text)  #  remove hashtag
    text=re.sub(r'@\S*','',text)  # remove user name 
    text=re.sub(r'https?:\/\/\S+','',text)  #remove hyperlinks
    text=re.sub('\d+','',text)  # remove digits
    text=re.sub(r'^\s+','',text)  # remove space in front of text 
    text=re.sub(r'\s+$','',text)  # remove space in tail text
    return text

In [ ]:
def stemming(text):
  ps = PorterStemmer()
  output = [ps.stem(word) for word in text if word not in stopwords]

  return output

In [ ]:
def process_data(text):
  clean_text = cleanText(text)
  # tokenized_text = word_tokenize(clean_text)
  tokenized_text = clean_text.split()
  output = stemming(tokenized_text)
  output = ' '.join(output)
  return output

In [ ]:
data['Clean_text'] = data['text'].apply(lambda x: process_data(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data.to_csv('clean_data.csv', index=False)

# **Prepare data**

In [ ]:
X = data['Clean_text']
Y = data['label']

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 141948)	0.012825013714838048
  (0, 141591)	0.050895451539178314
  (0, 141539)	0.01546275020476954
  (0, 141528)	0.010453171646042076
  (0, 140517)	0.04581590268279043
  (0, 140511)	0.09829625188875421
  (0, 140444)	0.03917614541910199
  (0, 140386)	0.011862997179343502
  (0, 139908)	0.016444633897399186
  (0, 139747)	0.03269220234393787
  (0, 138964)	0.023563231570149643
  (0, 138872)	0.01431728893381932
  (0, 138621)	0.017712908844678396
  (0, 138526)	0.01301546660224641
  (0, 138417)	0.030322646194194558
  (0, 138318)	0.013700979619280141
  (0, 138158)	0.021925887522710037
  (0, 138083)	0.012357401028964328
  (0, 137000)	0.02694938774597126
  (0, 136963)	0.03276735390718657
  (0, 135744)	0.022320747175859423
  (0, 134053)	0.0677506207072004
  (0, 133448)	0.029081752132003452
  (0, 133105)	0.034694817783828574
  (0, 132718)	0.018119521987882758
  :	:
  (20760, 7849)	0.025373005702984314
  (20760, 7843)	0.01619003639261705
  (20760, 7555)	0.045543041598931966
  (20760, 7085)	0.02

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=42)

# **Model**

In [ ]:
svm = SVC(kernel='rbf')
svm.fit(X_train, Y_train)

SVC()

In [ ]:
# accuracy score on the training data
X_train_prediction = svm.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9985549132947977


In [ ]:
lr = LogisticRegression()
lr.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
# accuracy score on the training data
X_train_prediction_ = lr.predict(X_train)
training_data_accuracy_ = accuracy_score(X_train_prediction_, Y_train)

print('Accuracy score of the training data : ', training_data_accuracy_)

Accuracy score of the training data :  0.973807803468208
